In [44]:
import pandas as pd
import numpy as np
import pymongo
from py2neo import Graph, Node, Relationship, NodeMatcher
import warnings

# Desabilitar o aviso temporariamente
warnings.filterwarnings("ignore", category=FutureWarning)

In [45]:
# estabelecer conexão com o servidor MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017")

# estabelecer conexão neo4j
graph = Graph("bolt://localhost:7687 ", auth=("neo4j", "password"))
matcher = NodeMatcher(graph)

In [46]:
mongo_database = client["Electronics_Store"]
collection_product = mongo_database["product"] 
collection_employees = mongo_database["employees"] 
collection_employees_archive = mongo_database["employees_archive"] 
collection_order_details = mongo_database["order_details"]
collection_store_users = mongo_database["store_users"] 
collection_cart_item = mongo_database["cart_item"] 
collection_shopping_session = mongo_database["shopping_session"] 
collection_order_items = mongo_database["order_items"] 
collection_addresses = mongo_database["addresses"]

### PRODUCT

In [47]:
product_json = collection_product.find()
for p in product_json:
    product_node = Node("PRODUCT", product_id=p["product_id"], product_name=p["product_name"], category_id=p["category_id"], sku=p["sku"], price=p["price"], discount_id=p["discount_id"], created_at_x=p["created_at_x"], last_modified=p["last_modified"], category_name=p["category_name"], discount_name=p["discount_name"], discount_desc=p["discount_desc"], discount_percent=p["discount_percent"], is_active_status=p["is_active_status"], created_at_y=p["created_at_y"], modified_at=p["modified_at"], quantity=p["quantity"], max_stock_quantity=p["max_stock_quantity"], unit=p["unit"])
    graph.create(product_node)

### EMPLOYEES

In [48]:
employees_json = collection_employees.find() 
for e in employees_json:
    employee_node = Node("EMPLOYEE", employee_id=e["employee_id"], first_name=e["first_name"], middle_name=e["middle_name"], last_name=e["last_name"], date_of_birth=e["date_of_birth"], department_id=e["department_id"], hire_date=e["hire_date"], salary=e["salary"], phone_number=e["phone_number"], email=e["email"], ssn_number=e["ssn_number"], manager_id=e["manager_id"], department_name=e["department_name"], manager_department_id=e["manager_department_id"], department_desc=e["department_desc"])
    graph.create(employee_node)
    

### EMPLOYEES ARCHIVE

In [49]:
employees_archive_json = collection_employees_archive.find()
for ea in employees_archive_json:
    employee_archive_node = Node("EMPLOYEE_ARCHIVE", event_date=ea["event_date"], event_type=ea["event_type"], user_name=ea["user_name"], old_employee_id=ea["old_employee_id"], old_first_name=ea["old_first_name"], old_middle_name=ea["old_middle_name"], old_last_name=ea["old_last_name"], old_date_of_birth=ea["old_date_of_birth"], old_department_id=ea["old_department_id"], old_hire_date=ea["old_hire_date"], old_salary=ea["old_salary"], old_phone_number=ea["old_phone_number"], old_email=ea["old_email"], old_ssn_number=ea["old_ssn_number"], old_manager_id=ea["old_manager_id"], new_employee_id=ea["new_employee_id"], new_first_name=ea["new_first_name"], new_middle_name=ea["new_middle_name"], new_last_name=ea["new_last_name"], new_date_of_birth=ea["new_date_of_birth"], new_department_id=ea["new_department_id"], new_hire_date=ea["new_hire_date"], new_salary=ea["new_salary"], new_phone_number=ea["new_phone_number"], new_email=ea["new_email"], new_ssn_number=ea["new_ssn_number"], new_manager_id=ea["new_manager_id"])
    
    graph.create(employee_archive_node)

### STORE USERS

In [50]:
store_users_json = collection_store_users.find()
for su in store_users_json:
    store_users_node = Node("STORE_USERS", user_id=su["user_id"], first_name=su["first_name"], middle_name=su["middle_name"], last_name=su["last_name"], phone_number=su["phone_number"], email=su["email"], username=su["username"], user_password=su["user_password"], registered_at=su["registered_at"])
    
    graph.create(store_users_node)

### ADDRESSES

In [51]:
addresses_json = collection_addresses.find()
for ad in addresses_json:
    addresses_node = Node("ADDRESSES", adress_id=ad["adress_id"], line_1=ad["line_1"], line_2=ad["line_2"], city=ad["city"], zip_code=ad["zip_code"], province=ad["province"], country=ad["country"])
    graph.create(addresses_node)

### ORDER DETAILS

In [52]:
order_details_json = collection_order_details.find()
for od in order_details_json:
    order_details_node = Node("ORDER_DETAILS", order_details_id=od["order_details_id"], user_id=od["user_id"], total=od["total"], payment_id=od["payment_id"], shipping_method=od["shipping_method"], delivery_adress_id=od["delivery_adress_id"], created_at=od["created_at"], modified_at=od["modified_at"], amount=od["amount"], provider=od["provider"], payment_status=od["payment_status"], payment_created_at=od["payment_created_at"], payment_modified_at=od["payment_modified_at"])
    graph.create(order_details_node)

    # RELATION ADDRESSES ORDER_DETAILS
    
    # vai buscar o node adresses
    addr_id = od["delivery_adress_id"]
    addr = matcher.match("ADDRESSES", adress_id=addr_id).first()

    addr_order_rel = Relationship(order_details_node, "delivery", addr)
    graph.create(addr_order_rel)

### SHOPPING SESSION

In [53]:
shopping_session_json = collection_shopping_session.find()
for ss in shopping_session_json:
    shopping_session_node = Node("SHOPPING_SESSION", session_id=ss["session_id"], user_id=ss["user_id"], created_at=ss["created_at"], modified_at=ss["modified_at"])
    graph.create(shopping_session_node)

### CART_ITEM

In [54]:
cart_item_json = collection_cart_item.find()
for ci in cart_item_json:
    # Vai buscar nodo product
    prod_id = ci["product_id"]
    prod = matcher.match("PRODUCT", product_id=prod_id).first()

    # Vai buscar nodo session
    session_id = ci["session_id"]
    session = matcher.match("SHOPPING_SESSION", session_id=session_id).first()

    cart_item_rel = Relationship(prod, "cart_item", session, cart_item_id=ci["cart_item_id"], session_id=ci["session_id"], product_id=ci["product_id"], quantity=ci["quantity"], created_at=ci["created_at"], modified_at=ci["modified_at"])

    
    graph.create(cart_item_rel)

### ORDER ITEMS

In [55]:
order_items_json = collection_order_items.find()
for oi in order_items_json:
    # Vai buscar nodo product
    prod_id = oi["product_id"]
    prod = matcher.match("PRODUCT", product_id=prod_id).first()

    # Vai buscar nodo session
    order_details_id = oi["order_details_id"]
    order_det = matcher.match("ORDER_DETAILS", order_details_id=order_details_id).first()

    order_items_rel = Relationship(order_det, "order_items", prod, order_items_id=oi["order_items_id"], order_details_id=oi["order_details_id"], product_id=oi["product_id"], created_at=oi["created_at"], modified_at=oi["modified_at"])
    graph.create(order_items_rel)